# Installing Modules

Please run the code below to install all the necessary modules that will be used in this application notebook.

In [1]:
!pip install pandas
!pip install flask
!pip install flask-ngrok
!pip install nltk

In [0]:
from flask import Flask,request
from flask_ngrok import run_with_ngrok
import nltk
import pandas as pd
import re

In [3]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def get_weighted_dictionary(words, stopwords):
  words_freq = {}
  for word in words:
    if word not in stopwords:
      if word not in words_freq:
        words_freq[word] = 1
      else:
        words_freq[word] += 1
  max_freq = max(words_freq.values())
  for key in words_freq.keys():
    words_freq[key] /= max_freq
  return words_freq

In [5]:
stopwords = nltk.corpus.stopwords.words("english")
word1 = ["Hello", "Yes", "Bye", "Hello", "Bye", "Rahul", "Bye", "Bye", "Jai"]
dict1 = get_weighted_dictionary(word1, stopwords)
dict1

{'Bye': 1.0, 'Hello': 0.5, 'Jai': 0.25, 'Rahul': 0.25, 'Yes': 0.25}

In [0]:
def get_sentence_score(sentences, word_freq, stopwords):
  sentences_score = []
  for sent in sentences:
    current_words = nltk.word_tokenize(sent)
    current_score = 0
    for word in current_words:
      if word not in stopwords:
        current_score += word_freq[word]
    sentences_score.append(current_score)
  return sentences_score

In [7]:
sent = ["Hello Rahul", "Yes", "Hello and Bye Rahul"]
get_sentence_score(sent, dict1, stopwords)

[0.75, 0.25, 1.75]

In [0]:
def get_summary(text, percentage = 0.3):
  sentences = nltk.sent_tokenize(text)
  stopwords = nltk.corpus.stopwords.words("english")
  words = nltk.word_tokenize(text)
  word_freq = get_weighted_dictionary(words, stopwords)
  sentences_score = get_sentence_score(sentences, word_freq, stopwords)
  data = pd.DataFrame({"sentence":sentences, "score":sentences_score})
  n = data.shape[0]
  top = data.sort_values(by = "score", ascending=False).iloc[0:int(n*percentage),0]
  return " ".join(top)

In [9]:
get_summary("Machine Learning is really cool. Machine Learning finds it application in statistical and data mining application. Machine Learning takes data and returns a prediction on that data. There are many cool applications of machine learning. Machine Learning has enabled data scientists to find hidden patterns in data.")

'Machine Learning has enabled data scientists to find hidden patterns in data.'

In [0]:
app = Flask(__name__)
run_with_ngrok(app)

In [0]:
html_style = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Text Summarizer</title>
    <style>
        *,html,body{
            margin:0px;
            padding: 0px;
        }
        #header{
            background: url(https://dtsvkkjw40x57.cloudfront.net/1350xnull/8160/uploads/2ab85f0f-bc62-417f-8a33-a064535a6ebd.jpg);
            opacity: 0.9;
            height: 300px;
            width: 100%;
            margin: 0px 0px 10px 0px;
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
        }

        #header h1 {
            border: 0px solid #F49200;
            background-color: #F49200;
            border-radius: 5px;
        }
        #form-input,#output
        {
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
        }
        #output{
          display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            width:600px;
            margin:auto;
        }
        textarea{
            width: 500px;
            height: 100px;
            border: 1px solid orange;
            resize: none;
        }
        input{
            background-color: orange;
            color: white;
            border:none;
            outline: none;
            border-radius: 5px;
            width:70px;
            height: 30px;
        }
        form,#output{
            margin-top: 10px;
        }
        #output p{
            border: 1px solid orange;
            border-radius: 5px;
            padding: 2px 2px 0px 2px;
        }

        a{
            margin-top: 10px;
        }
        #desc{
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            margin-top: 30px;
        }
        ul{
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div id="header">
        <h1 style="color: white;">Welcome to Text Summarization Application</h1>
    </div>
'''

In [0]:
not_posted = '''
<div id="form-input">
        <h3>Please Enter Your Text Here....</h3>
        <form method="POST">
            <textarea id = "text" name="text" placeholder="Enter your text here..."></textarea>
            <input type="submit" id="submit">
        </form>
    </div>'''

In [0]:
posted = lambda x : '''
<div id="output">
        <h3>Here is the summarized version of the text you provided....</h3>
        <p>''' + x + '''</p>
        <a href="/">Go Back</a>
    </div>
'''

In [0]:
bottom = '''
<div id = "desc">
        <h2>Here is a Little description on how the algorithm works...</h2>
        <ul>
            <li>This uses one of the most powerful algorithms in Natural Language Processing for Summarization Called TextRank.</li>
            <li>It is the same algorithm which was used by Google to rank the search results to show you the best results.</li>
            <li>You enter the text in the Input and the text data is sent to the server.</li>
            <li>One the server receives the text, we have a function which will receive the entered text and it is going to apply Text Rank algorithm.</li>
            <li>Then, that function is going to produce a summarized version of the entered text.</li>
        </ul>
    </div>
</body>
</html>
'''

In [0]:
@app.route("/", methods = ["GET", "POST"])
def index():
    if request.method == "POST":
        required_text = request.form.get("text")
        summarized_text = get_summary(required_text, percentage = 0.3)
        request.form.data = ""
        return html_style + posted(summarized_text) + bottom
    return html_style + not_posted + bottom

## Running the application

Finally, once our app is ready and all the routes have been created, we can now run our application by calling the `run()` method on the app variable and we are ready to use this application..


** *NOTE: Once we run the application, because we have used app with flask-ngrok, it is going to point us at two links. One which will always be http://localhost:5000/ and another which will be always different but will include the name ngrok in the URL so that you can get to know that this is a ngrok URL. With this URL, your application will be temporarily hosted on a ngrok platform and you can access it from there. This is useful when your are not running this notebook on your local system, otherwise, localhost URL will work just fine.* **

In [16]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3a0fae4a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
